<a href="https://colab.research.google.com/github/Duncan1738/Powered-Image-Colorization/blob/main/colorize_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import argparse

# Load Pretrained Model Files
PROTOTXT_PATH = "colorization_deploy_v2.prototxt"
MODEL_PATH = "colorization_release_v2.caffemodel"
POINTS_PATH = "pts_in_hull.npy"

# Load the model
net = cv2.dnn.readNetFromCaffe(PROTOTXT_PATH, MODEL_PATH)
pts_in_hull = np.load(POINTS_PATH)

# Load cluster centers as 1x1 convolution kernel
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts_in_hull.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

def colorize_image(image_path, output_path="colorized.png"):
    # Load Image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Resize and normalize
    L = image[:, :, 0]  # Extract L channel
    L = cv2.resize(L, (224, 224))
    L -= 50  # Normalize

    # Process through DNN
    net.setInput(cv2.dnn.blobFromImage(L))
    ab = net.forward()[0, :, :, :].transpose((1, 2, 0))

    # Resize and merge channels
    ab = cv2.resize(ab, (image.shape[1], image.shape[0]))
    colorized = np.concatenate((image[:, :, 0][:, :, np.newaxis], ab), axis=2)

    # Convert back to BGR
    colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
    colorized = (np.clip(colorized, 0, 255)).astype("uint8")

    # Save output
    cv2.imwrite(output_path, colorized)
    print(f"✅ Image saved to {output_path}")

# Run script with command-line argument
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Colorize a black-and-white image")
    parser.add_argument("image_path", help="Path to the black and white image")
    args = parser.parse_args()

    colorize_image(args.image_path)
